In [2]:
import pandas as pd
from datetime import datetime
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import word_preprocessing as wpp
from textblob_de import TextBlobDE as TextBlob
import nltk

In [10]:
data = pd.read_excel("../data/data_vdss.xlsx")
data.head()

,id,pubtime,medium_code,medium_name,rubric,regional,doctype,doctype_description,language,char_count,dateline,head,subhead,content_id,content
0,47612274,2022-09-20 11:11:09+02,BLIO,blick.ch,Wirtschaft - Schweden beschliesst starke Leitz...,NaN,WWE,Online medium,de,1019.0,NaN,Schweden beschliesst starke Leitzinserhöhung,NaN,00000f5a-b2d4-61df-b724-c788cd754a75,<tx><ld><p>Die schwedische Notenbank stemmt si...
1,47624348,2022-09-21 14:18:32+02,NNTA,tagesanzeiger.ch,Russland,NaN,WWE,Online medium,de,4545.0,NaN,Teilmobilmachung in Russland: Wie Ramsan Kadyr...,NaN,0000582e-aa41-3899-a99c-27459ab7aaf5,<tx><ld><p>Der tschetschenische Herrscher hatt...
2,46006821,2022-04-08 12:59:57+02,LUZO,luzernerzeitung.ch,NaN,NaN,WWE,Online medium,de,4085.0,NaN,US-Wasserversorger mit starker Rendite: +137.8...,NaN,0000cab5-ed36-2ba6-7676-f96162bc756d,<tx><ld><p>Im Börsen-Blog «Money» porträtiert ...
3,47381556,2022-08-28 00:00:00+02,NZZS,NZZ am Sonntag,Mensch und Medizin,NaN,PJO,Regional weekly newspaper,de,668.0,NaN,News,NaN,00013bdd-1d8a-6673-bb9d-f893cdff87be,<tx><zt>Medizincannabis und Nikotin gehen Hand...
4,47092592,2022-07-27 16:06:03+02,NZZO,nzz.ch,wirtschaft,NaN,WWE,Online medium,de,8386.0,NaN,"Erdgas ist sündteuer, doch die Geschäfte blühe...",NaN,00014930-8365-3570-25af-d9af2c074f2c,<tx><ld><p>Der deutsche Chemiekonzern BASF ver...


In [69]:
# Datum bereinigen & redundante Spalten löschen
data['datum'] = data['pubtime'].str.extract(r'(\d{4}-\d{2}-\d{2})')

del data['pubtime']
del data['medium_code']
del data['regional']
del data['doctype']
del data['language']
del data['char_count']
del data['dateline']
del data['subhead']
del data['content_id']

KeyError: 'pubtime'

In [51]:
data.head()

,id,medium_name,rubric,doctype_description,head,content,datum
0,47612274,blick.ch,Wirtschaft - Schweden beschliesst starke Leitz...,Online medium,Schweden beschliesst starke Leitzinserhöhung,<tx><ld><p>Die schwedische Notenbank stemmt si...,2022-09-20
1,47624348,tagesanzeiger.ch,Russland,Online medium,Teilmobilmachung in Russland: Wie Ramsan Kadyr...,<tx><ld><p>Der tschetschenische Herrscher hatt...,2022-09-21
2,46006821,luzernerzeitung.ch,NaN,Online medium,US-Wasserversorger mit starker Rendite: +137.8...,<tx><ld><p>Im Börsen-Blog «Money» porträtiert ...,2022-04-08
3,47381556,NZZ am Sonntag,Mensch und Medizin,Regional weekly newspaper,News,<tx><zt>Medizincannabis und Nikotin gehen Hand...,2022-08-28
4,47092592,nzz.ch,wirtschaft,Online medium,"Erdgas ist sündteuer, doch die Geschäfte blühe...",<tx><ld><p>Der deutsche Chemiekonzern BASF ver...,2022-07-27


In [73]:
data['head'][data['datum'] == '2022-07-07']

973       Nevil Marti ist der jüngste Segelflugpilot der...
1028      Tennis: Rafael Nadal muss für Wimbledon-Halbfi...
1460      Unterwegs im Privatzug: Ums Ankommen geht es h...
2279        Das sind die Spitzenpaarungen am Rigi Schwingen
2856              Wir müssen mehr über unseren Schutz reden
                                ...                        
205121    Openair Frauenfeld 2022: Alle Informationen im...
205150    Bestände erholen sich: Grosse Finnwal-Gruppen ...
205151    Bestände erholen sich: Grosse Finnwal-Gruppen ...
205306    Boris Johnsons Abtrittsrede: Der «kleine Trump...
205349    KURZMELDUNGEN - Sport: Norwegerinnen gewinnen ...
Name: head, Length: 617, dtype: object

In [20]:
# Headlines aufbereiten mit Klasse word_preprocessing
# testweise nur 100 erste Zeilen

preprocesser = wpp.headline_preprocessing()
data['titel_preprocessed'][:100] = preprocesser.preprocess_headlines(data['titel'][:100])
data

C:\Users\linus\AppData\Local\Temp\ipykernel_1156\1560632050.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['titel_preprocessed'][:100] = preprocesser.preprocess_headlines(data['titel'][:100])


,beitragid,titel,mediumid,anzahlzeichen,Inhalt,datum,Sentiment,titel_preprocessed
0,21319059,Über das Ziel hinausgeschossen,1,3178,"André Müller, amü\nWerde digitaler, sonst steh...",2017-10-25,0.150000,Ziel hinausgeschossen
1,21312273,Gewagte Schätzungen für die SKA 2.0,2,3474,Gewagte Schätzungen für die SKA 2.0 ...,2017-10-21,0.175714,Gewagt Schätzung SKA 20
2,20946237,"Der Spion, den wir liebten",4,7259,Jean-Claude Galli\nIn den letzten Jahren ist e...,2017-05-24,0.210423,Spion lieben
3,21826660,Endlich Hilfe gegen Migräne!,4,1041,Neue Spritze bald auf dem Markt 1 Million Schw...,2018-06-29,-0.200000,Hilfe Migran
4,21820335,Swiss Life übernimmt deutsche Immobilienfirma,1,1362,(sda) · Die Swiss Life baut ihre Stellung am d...,2018-06-26,0.231818,Swiss Life übernehmen deutsch Immobilienfirma
...,...,...,...,...,...,...,...,...
35892,22561096,kein Titel,946,1073,"Ein explosiver Vorwurf: «Was Roche tut, ist ei...",2019-02-09,NaN,NaN
35893,23150512,Schweizer Händler profitieren von Waldbränden ...,950,7467,In einem unscheinbaren Gebäude am Chemin de No...,2019-09-08,NaN,NaN
35894,23009642,Swiss Life begrüsst Pläne zu tieferem Umwandlu...,946,2113,"Die Vorschläge, welche die Sozialpartner und d...",2019-07-15,NaN,NaN
35895,23209865,«Die Reputation der Credit Suisse hat gelitten»,946,4153,Die Bespitzelungsaffäre bei der Credit Suisse ...,2019-10-01,NaN,NaN


In [14]:
def get_topic_sentiments(headlines):
    headline_blob = TextBlob(headlines)
    return headline_blob.sentiment.polarity

# Sentiment-Analyse
# testweise nur 100 erste Zeilen

data['Sentiment'] = data['Inhalt'][0:100].apply(lambda x: get_topic_sentiments(x))

In [15]:
data

,beitragid,titel,mediumid,anzahlzeichen,Inhalt,datum,Sentiment
0,21319059,Über das Ziel hinausgeschossen,1,3178,"André Müller, amü\nWerde digitaler, sonst steh...",2017-10-25,0.150000
1,21312273,Gewagte Schätzungen für die SKA 2.0,2,3474,Gewagte Schätzungen für die SKA 2.0 ...,2017-10-21,0.175714
2,20946237,"Der Spion, den wir liebten",4,7259,Jean-Claude Galli\nIn den letzten Jahren ist e...,2017-05-24,0.210423
3,21826660,Endlich Hilfe gegen Migräne!,4,1041,Neue Spritze bald auf dem Markt 1 Million Schw...,2018-06-29,-0.200000
4,21820335,Swiss Life übernimmt deutsche Immobilienfirma,1,1362,(sda) · Die Swiss Life baut ihre Stellung am d...,2018-06-26,0.231818
...,...,...,...,...,...,...,...
35892,22561096,kein Titel,946,1073,"Ein explosiver Vorwurf: «Was Roche tut, ist ei...",2019-02-09,NaN
35893,23150512,Schweizer Händler profitieren von Waldbränden ...,950,7467,In einem unscheinbaren Gebäude am Chemin de No...,2019-09-08,NaN
35894,23009642,Swiss Life begrüsst Pläne zu tieferem Umwandlu...,946,2113,"Die Vorschläge, welche die Sozialpartner und d...",2019-07-15,NaN
35895,23209865,«Die Reputation der Credit Suisse hat gelitten»,946,4153,Die Bespitzelungsaffäre bei der Credit Suisse ...,2019-10-01,NaN
